In [107]:
import tensorflow as tf
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import os
import copy
from gensim.models import KeyedVectors

In [108]:
# Conf
langs = ['eng', 'deu', 'hun']

Read embeddings and indices

In [109]:
fold = '/home/eszti/projects/dipterv/notebooks/panlex/train_try/t0'

d_models = dict()
for l in langs:
    fn = os.path.join(fold, '{}.vec'.format(l))
    model = KeyedVectors.load_word2vec_format(fn, binary=False)
    d_models[l] = model

In [110]:
# test
print(d_models['hun'].syn0.shape)
print(d_models['hun'].index2word)

(4, 300)
['kutya', 'macska', 'nap', 'hold']


Read words pairs from tsv

In [111]:
def read_word_pairs_tsv(fn, id1, id2):
    with open(fn) as f:
        lines = f.readlines()
        data = [(line.split()[id1], line.split()[id2]) for i, line in enumerate(lines) if i > 0]
    return data

In [112]:
fold = '/home/eszti/projects/dipterv/notebooks/panlex/train_try/t0'
id1 = 2
id2 = 3
# Dict for word pairs
d_wps = dict()
done = set()
for lang1 in langs:
    for lang2 in langs:
        lang_pair = tuple(sorted([lang1, lang2]))
        if lang1 == lang2 or lang_pair in done:
            continue
        done.add(lang_pair)
        l1 = lang_pair[0]
        l2 = lang_pair[1]
        fn = os.path.join(fold, '{0}_{1}.tsv'.format(l1, l2))
        data = read_word_pairs_tsv(fn, id1, id2)
        d_wps[lang_pair] = data

In [113]:
d_wps

{('deu', 'eng'): [('katze', 'cat'),
  ('hund', 'dog'),
  ('mond', 'moon'),
  ('sonne', 'sun')],
 ('deu', 'hun'): [('hund', 'kutya'),
  ('katze', 'macska'),
  ('mond', 'hold'),
  ('sonne', 'nap')],
 ('eng', 'hun'): [('cat', 'macska'),
  ('dog', 'kutya'),
  ('sun', 'nap'),
  ('moon', 'hold')]}

Calculate precision

In [114]:
def precision():
    # Todo
    pass

Doing training

In [115]:
def train(langs, d_embs, d_idxs, d_wordpairs, dim, iters, lr=0.3):
    nb_langs = len(langs)

    # Init graphs
    graph = tf.Graph()
    with graph.as_default():
        # TF variables 
        # Placeholder for 2 words
        tf_w1 = tf.placeholder(tf.float64, shape=[None, dim])
        tf_w2 = tf.placeholder(tf.float64, shape=[None, dim])
        # Placeholder for indexing the T matrix
        tf_idx_l1 = tf.placeholder(tf.int32)
        tf_idx_l2 = tf.placeholder(tf.int32)
        # Translation matrices
        tf_T = tf.Variable(tf.truncated_normal([nb_langs, dim, dim]))

        # Loss
        loss = tf.matmul(tf.matmul(tf_w1, tf_T[tf_idx_l1]), tf.transpose(tf.matmul(tf_w2, tf_T[tf_idx_l2])))

        optimizer = tf.train.GradientDescentOptimizer(lr).minimize(loss)

    with tf.Session(graph=graph) as session:
        tf.global_variables_initializer().run()

        for i in range(iters):
             # Run the computations
            _, l, T = session.run([optimizer, loss, tf_T])
            # Todo : como funciona?
            session.run(train_step, feed_dict={x: batch_x, y: batch_y})

In [145]:
m1 = copy.deepcopy(d_models['eng'])
m2 = copy.deepcopy(m1)

W = copy.deepcopy(m2.syn0)
T = np.random.rand(300,300)
# print(W)
# print(T)
m2.syn0 = np.dot(W, T)
# print(m2.syn0)
print(m1.most_similar('dog'))
print(m2.most_similar('dog'))

[('cat', 0.6380517482757568), ('moon', 0.2543850839138031), ('sun', 0.1620684415102005)]
[('cat', 0.5462808609008789), ('moon', 0.5094023942947388), ('sun', -0.38269948959350586)]
